In [1]:
### Hydrological event identification based on hydrograph

In [2]:
### import packages
# general packages
import os
import datetime as dt
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt

# packages for speical usage
from hydrotools.nwis_client import IVDataService

import itertools
from pandas import Grouper
import operator

In [13]:
startDate = "2020-01-01"
endDate = "2020-12-31"

In [9]:
# ### Directory and file paths
# os.chdir("~/Dropbox/PhD_Dissertation/CH3_WRFHydro_project/Script/")    # set current working directory

# # directories
# dir_result = "../Results/event_identification/"

# files
file_site_meta = "./Data/domainMeta_detail.csv"

In [5]:
### Conversions
cfs2cms = 0.02832
sqmi2sqkm = 2.58999

In [294]:
def event_identify(obs_gage, dir_output, gage_id, thrld_min, thrld_jump=80, thrld_flat=75, no_flat=3, thrld_duration='6H',
                  plot = True, plot_start=None, plot_end=None):
    '''
    Identify events based on the hydrograph (raising and falling limb identification) + applied thresholds.
    obs_gage: hourly streamflow time series with `datetime` (UTC) as the index and `value` as the streamflow (cms).
    dir_output: output directory.
    gage_id: string. for creating the output file.
    thrld_min: minimum peak streamflow (cms) to consider an event as identified event.
    thrld_jump: percentile of changes/shifts (in an hour) to identify a "jump" (to prevent tiny jumps from unstable/sensitive streamflow). Default: 80.
    thrld_flat: percentile of changes/shifts (in an hour) to identify "flat" area.
    no_flat: the number of timesteps for consecutively counting as 'flat'.
    thrld_duration: the threshold for the event duration. Default - at least longer than 6H.
    plot: if you want a plot or not
    plot_start: the starting time of the plot. default is the begining of the data
    plot_end: the ending time of the plot. default is the last data point.
    '''
    # determine the "jump"
    data = obs_gage.value-obs_gage.value.shift(1)
    obs_gage.loc[:,'jump'] = data>np.percentile([x for x in data if x>0], thrld_jump)  # need to have this to prevent getting tiny jump from unstable streamflow

    ## Looking for the "flat" area (the first of flat should be the end of falling limb): negative changes, streamflow varies within threld_flat percentile of change/shifts
    obs_gage.loc[:, 'flat'] = (data<=0.01)&(abs(data)<=(np.percentile([abs(x) for x in data if x<0], thrld_flat)))

    ### Extract the indices of jump and flat
    # indices for 'jumps'
    count_dups = [[i, sum(1 for i in group)] for i, group in groupby(obs_gage['jump'])]
    results = itertools.accumulate(list(map(operator.itemgetter(1), count_dups)), operator.add)
    idx = [each for each in results]

    # Add the accumulated index 
    for i in range(len(count_dups)):
        count_dups[i].append(idx[i])

    idx_jump = [idx[2]-idx[1] for idx in count_dups if (idx[0]==True)]

    # indices for the 'flats'
    count_dups = [[i, sum(1 for i in group)] for i, group in groupby(obs_gage['flat'])]
    results = itertools.accumulate(list(map(operator.itemgetter(1), count_dups)), operator.add)
    idx = [each for each in results]

    # Add the accumulated index 
    for i in range(len(count_dups)):
        count_dups[i].append(idx[i])

    # flat and decay. both needs to have at least three consecutive flat/deay
    idx_flat_start = [idx[2]-idx[1]-1 for idx in count_dups if (idx[0]==True)&(idx[1]>=no_flat)]  #start of flat
    idx_flat_end = [idx[2]-1 for idx in count_dups if (idx[0]==True)&(idx[1]>=no_flat)]   # end of flat

    obs_gage.loc[:,'flat'] = False
    obs_gage.loc[:,'flat'].iloc[idx_flat_start]=True

    ### List the event
    from datetime import datetime
    start = []
    end = []

    ## Found the first timestep of "flat" after a jump
    for i in range(len(idx_jump)):
        start_tmp = obs_gage.iloc[idx_jump].index[i]
        start.append(min(obs_gage.iloc[idx_flat_end].index, key=lambda x: (x>start_tmp, abs(start_tmp-x))))
        end.append(min(obs_gage.iloc[idx_flat_start].index, key=lambda x: (x<start_tmp, abs(x-start_tmp))))

    ## Create an empty list for storing event info
    event_list = pd.DataFrame({
                    'start': start,
                    'end': end,
                    'Qmax': np.nan,
                    'Qmin': np.nan,
                    'Qrange': np.nan,
                    'Qmean': np.nan,
                    'Qmedian': np.nan,
                    'Qstd': np.nan,
                    'Qvar': np.nan,
                    'Qskew': np.nan,
                    'Qkurt': np.nan,
                    'Qtotal': np.nan,
                    'Qnpeak': np.nan,
                    'Qlen': np.nan,
                    'Qtrange': np.nan,
                 })
    
    ## Drop the event that is within another event
    event_list = event_list.drop_duplicates(subset='end',keep="first")
    event_list = event_list.drop_duplicates(subset='start',keep="last")

    ## Extend the event starting/ending time for 3 hour, respectively
    event_list.loc[:,'start'] = event_list['start'] - dt.timedelta(hours=3)
    event_list.loc[:,'end'] = event_list['end'] + dt.timedelta(hours=3)

    ### Discard the event that is shorter than thrld_duration
    event_list = event_list[(event_list.end-event_list.start)>=thrld_duration]

    ## Mark the event_list back to the obs_gage
    obs_gage.loc[:,'event'] = False

    for i in range(len(event_list)):
        start = event_list.start.iloc[i]
        stop = event_list.end.iloc[i]
        Qevent = obs_gage.loc[start:stop]
        ### filter out if the event max. larger than the threshold
        if Qevent.max().value>=thrld_min:

            # Calculate event statistics
            ## magnitude related
            Qmax = Qevent.value.max()    #cms, maximum Q
            Qmin = Qevent.value.min()    #cms, minmum Q
            Qrange = Qmax - Qmin    #cms, range of Q
            Qmean = Qevent.value.mean()    # cms, mean Q
            Qmedian = Qevent.value.median()    # cms, median Q
            Qstd = Qevent.value.std()    # cms, standard deviation
            Qvar = Qevent.value.var()    # cms, variance
            Qskew = Qevent.value.skew()    # skewness
            Qkurt = Qevent.value.kurt()    # kurt index

            Qtotal = Qevent.value.sum()*3600  #m^3, total event discharge
            Qnpeak = Qevent.jump.sum() #the number of peaks in the event

            ## time related
            Qlen = len(Qevent) # event duration (time interval must be "hour" here), event length
            Qtrange = int((Qevent.loc[Qevent.value==Qmax,'value'].index - start).total_seconds()[0]/3600) - 3 # hour, time to peak (from first Q rise, not starts of event)

            ## mark the event
            obs_gage.loc[start:stop,'event'] = True

            ## write the statistics to the event_list
            event_list.iloc[i] = [start, stop, Qmax, Qmin, Qrange, Qmean, Qmedian, Qstd, Qvar, Qskew, Qkurt, Qtotal, Qnpeak, Qlen, Qtrange]

    
    # Tidy up the dataframes
    event_list.dropna(inplace=True)
    obs_gage.drop(['jump','flat'], axis=1, inplace=True)
    obs_gage.columns = ['Q_cms', 'event']
    obs_gage.index.name = 'Datetime'

    event_list.to_csv(dir_output + 'event_list_' + gage_id + '.csv', index=False)
    obs_gage.to_csv(dir_output + 'obs_event_marked_' + gage_id + '.csv', index=True)

    ## Plot
    if plot==True:
        if plot_start is None:
            plot_start = np.nanmin(obs_gage.index)
        if plot_end is None:
            plot_end = np.nanmax(obs_gage.index)

        obs_gage_trim = obs_gage.loc[plot_start:plot_end]

        obs_gage_trim.loc[obs_gage_trim.event==False,'event'] = 'blue'
        obs_gage_trim.loc[obs_gage_trim.event==True, 'event'] = 'orange'

        plt.subplots(figsize=(20,8))
        plt.scatter(obs_gage_trim.reset_index()['Datetime'], 
                    y=obs_gage_trim.reset_index()['Q_cms'], 
                    c=obs_gage_trim.reset_index()['event'], s=10)

        plt.ylabel('Discharge (cms)', labelpad=10, fontsize=24)
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)

        plt.savefig(dir_output + 'plot_events_' + gage_id + '.jpg', dpi=300)
    plt.close()


In [10]:
### Main code -------------------------------
### Read static files
site_meta = pd.read_csv(file_site_meta)

In [11]:
site_meta.site_no.values

array([16211600, 16213000, 16229000, 16241600, 16249000, 16274100,
       16294900, 16301050, 16330000, 16345000])

In [16]:
# Retrieve streamflow observations 
service = IVDataService()
observations = service.get(
    sites=[16211600, 16213000], 
    startDT=startDate, 
    endDT=endDate)

In [17]:
observations

,value_time,variable_name,usgs_site_code,measurement_unit,value,qualifiers,series
0,2020-01-01 00:00:00,streamflow,16211600,ft3/s,0.92,['A'],0
1,2020-01-01 00:15:00,streamflow,16211600,ft3/s,0.83,['A'],0
2,2020-01-01 00:30:00,streamflow,16211600,ft3/s,0.83,['A'],0
3,2020-01-01 00:45:00,streamflow,16211600,ft3/s,0.83,['A'],0
4,2020-01-01 01:00:00,streamflow,16211600,ft3/s,0.83,['A'],0
...,...,...,...,...,...,...,...
137705,2020-12-30 23:40:00,streamflow,16213000,ft3/s,22.40,['A'],0
137706,2020-12-30 23:45:00,streamflow,16213000,ft3/s,23.00,['A'],0
137707,2020-12-30 23:50:00,streamflow,16213000,ft3/s,23.00,['A'],0
137708,2020-12-30 23:55:00,streamflow,16213000,ft3/s,23.00,['A'],0


In [18]:
# Drop extra columns to be more efficient
obs = observations[[
    'usgs_site_code', 
    'value_time', 
    'value'
    ]]

# Check for duplicate time series, keep first by default
obs = obs.drop_duplicates(
    subset=['usgs_site_code', 'value_time']
    )

# Resample to hourly, keep first measurement in each 15-min bin
obs = obs.groupby([
    'usgs_site_code',
    Grouper(key='value_time', freq='1H')
    ]).first() ### Note, ffill will also fill those no-data.

# convert cfs to cms
obs = obs*cfs2cms

/var/folders/pn/p22nfm7d03l25m6v68vhscv80000gn/T/ipykernel_951/3352395016.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  obs = obs.groupby([


In [19]:
obs

value
usgs_site_code value_time                   
16211600       2020-01-01 00:00:00  0.026054
               2020-01-01 01:00:00  0.023506
               2020-01-01 02:00:00  0.023506
               2020-01-01 03:00:00  0.023506
               2020-01-01 04:00:00  0.023506
...                                      ...
16213000       2020-12-30 20:00:00  0.651360
               2020-12-30 21:00:00  0.651360
               2020-12-30 22:00:00  0.651360
               2020-12-30 23:00:00  0.634368
               2020-12-31 00:00:00  0.634368

[17522 rows x 1 columns]

In [296]:
## identify hydrological event 
# gage = site_meta.site_no[0]
# gage_id = str(gage)
# obs_gage = obs.loc[gage_id].copy()
# obs_gage = obs_gage.tz_localize('UTC')
### Manually fix the issue of the Waikele gage flown away =============================
gage = site_meta.site_no[1]
gage_id = str(gage)
obs_gage = obs.loc[gage_id].copy()
obs_gage.loc[(obs_gage.index >= dt.datetime(2016,7,22)) &
         (obs_gage.index <= dt.datetime(2016,7,29)) &
         (obs_gage.value > 60)] = np.nan
obs_gage = obs_gage.tz_localize('UTC')
### ===================================================================================
dir_output = dir_result
thrld_min = site_meta.loc[site_meta.site_no==gage, 'Q5_cms'].values[0]
thrld_jump = 80
thrld_flat = 75  # original was 50

event_identify(obs_gage, dir_output, gage_id, thrld_min)

/opt/miniconda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [297]:
## identify hydrological event
for i in range(2,10):
    gage = site_meta.site_no[i]
    gage_id = str(gage)
    obs_gage = obs.loc[gage_id].copy()
    obs_gage = obs_gage.tz_localize('UTC')
    dir_output = dir_result
    thrld_min = site_meta.loc[site_meta.site_no==gage, 'Q5_cms'].values[0]
    thrld_jump = 80
    thrld_flat = 75  # original was 50

    event_identify(obs_gage, dir_output, gage_id, thrld_min)

/opt/miniconda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/miniconda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/miniconda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

### Event statistics for each watershed

In [298]:
site_meta.loc[:,'drain_area_va']=site_meta['drain_area_va']*sqmi2sqkm

In [299]:
event_stat = pd.DataFrame({
                'site_no': site_meta.site_no,
                'site_name': site_meta.station_nm.values,
                'nQmax_mean': np.nan,
                'nQmin_mean': np.nan,
                'nQrange_mean': np.nan,
                'nQmean_mean': np.nan,
                'nQmedian_mean': np.nan,
                'nQstd_mean': np.nan,
                'nQvar_mean': np.nan,
                'Qskew_mean': np.nan,
                'Qkurt_mean': np.nan,
                'Qtotal_per_hr': np.nan,
                'Qnpeak_mean': np.nan,
                'Qlen_mean': np.nan,
                'Qtrange_mean': np.nan,
                'count': np.nan
})

event_stat.set_index(['site_no'], inplace=True)

In [300]:
# gage = site_meta.site_no[0]

i=0
for gage in site_meta.site_no:
    event = pd.read_csv(dir_result + 'event_list_' + str(gage) + '.csv', index_col=0)
    drain_area = site_meta.drain_area_va[site_meta.site_no==gage].values[0]
    event.iloc[:,2:9] = event.iloc[:,2:9]/drain_area
    event_stat.iloc[i,1:14] = event.mean(numeric_only=True).values
    event_stat.loc[gage,'Qtotal_per_hr'] = event.Qtotal.sum()/event.Qlen.sum()
    event_stat.loc[gage,'count'] = len(event)
    i=i+1

In [301]:
event_stat.to_csv(dir_result + 'event_list_summary.csv', index=True)